## Практика по SQL

### Знакомство с данными

Мы будем работать с набором данных «Книги». Он включает две таблицы, хранящиеся в схеме other, — `other.books` и `other.book_orders`.

<img src=sql_img1.png>

### Задачи

`Задание 09-1`

Составьте список книжных новинок. Новинками считаются все книги за последние пять лет.

Необходимые данные:

* название книги;
* год издания;
* автор;
* жанр.

Вывод отсортируйте по названиям книг.

In [ ]:
select
    book_name,
    publishing_year,
    author,
    genre
from 
    other.books
where 
    publishing_year > 2015
order by 1

`Задание 09-2`

Посмотрим на полученные данные.

Что-то подозрительно много книг — для рекламного буклета это явно перебор. Что будем делать? Потихоньку избавляться от лишнего.

Скажем, там есть книги без названия? Давайте их исключим из списка.

In [ ]:
select
    book_name,
    publishing_year,
    author,
    genre
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
order by 1

`Задание 09-3`

Теперь нам надо как-то урезать количество оставшихся книг.

Может, по рейтингу автора? Неплохо, только мы не знаем, какие категории авторов у нас есть. Давайте выясним это.

Выберите значения рейтинга авторов, имеющиеся в нашей базе. Отсортируйте вывод по алфавиту.

In [ ]:
select
    book_name,
    publishing_year,
    author,
    genre
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
order by 1, 5

`Задание 09-4`

Возьмём для рекламного буклета только книги отличных авторов!

Оставьте в выборке новых книг только авторов с рейтингом `'Excellent'`.

In [ ]:
select
    book_name,
    publishing_year,
    author,
    genre
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
and author_rating = 'Excellent'
order by 1

`Задание 09-5`

Добавьте в имеющуюся выборку известных авторов (со значением рейтинга `'Famous'`).

In [ ]:
select
    book_name,
    publishing_year,
    author,
    genre
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
and (author_rating = 'Excellent' or author_rating = 'Famous')
order by 1

`Задание 09-6`

Определите, сколько книг из выборки для рекламы попадает в каждую категорию рейтинга автора.

Нам понадобятся следующие данные:

* рейтинг автора (`author_rating`);
* количество книг (`cnt`).

Сортировка, как всегда, по алфавиту.

In [ ]:
select
    --book_name,
    --publishing_year,
    --author,
    --genre
    author_rating,
    count(author_rating) as cnt 
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
--and (author_rating = 'Excellent' or author_rating = 'Famous')
group by 1
order by 1

`Задача 09-7`

Выбираем книги с рейтингом автора отличный (`Excellent`), известный (`Famous`) и новый (`Novice`). И в конце добавим строку об общем количестве книг.

В выборке нас по-прежнему интересуют

* название книги,
* год издания,
* автор,
* жанр.

В итоге у вас должна получиться выборка следующего вида:

<img src=sql_img2.png>

In [ ]:
(select
    book_name,
    publishing_year,
    author,
    genre
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
and (author_rating = 'Excellent' or author_rating = 'Famous' or author_rating = 'Novice')
order by 1)
union all
(select
    'Total',
    count(*),
    null,
    null
from 
    other.books
where 
    publishing_year > 2015
and book_name is not null
and (author_rating = 'Excellent' or author_rating = 'Famous' or author_rating = 'Novice')
)

`Задание 09-8`

Для начала выберите всю информацию о заказах книг, выпущенных не более 10 лет назад. Отсортируйте заказы по дате в обратном порядке.

In [ ]:
select * 
from other.book_orders o 
join other.books b on b.book_id=o.book_id 
where b.publishing_year > 2010
order by o.order_date DESC

`Задание 09-9`

Теперь оставим в выборке только заказы от 2019 года и позднее.

In [ ]:
select * 
from other.book_orders o 
join other.books b on b.book_id=o.book_id 
where b.publishing_year > 2010
and extract(YEAR FROM o.order_date)>=2019
order by o.order_date DESC

`Задание 09-10`

Чтобы понять, заказы за какой период у нас есть, определите дату последнего заказа.

In [ ]:
select
    order_date
from other.book_orders
order by 1 desc
limit 1

`Задание 09-11`

Посчитайте общее количество заказов за каждый месяц (`month`). Отсортируйте вывод по месяцам в обратном порядке.

In [ ]:
select 
    extract(month from order_date),
    count(extract(month from order_date))
from other.book_orders o 
join other.books b on b.book_id=o.book_id 
where b.publishing_year > 2010
and extract(YEAR FROM o.order_date)>=2019
group by 1
order by 1 desc

`Задание 09-12`

Добавьте в предыдущий запрос подсчёт количества разных книг (`cnt_dist`), заказанных в каждом месяце.

In [ ]:
select 
    extract(month from order_date),
    count(extract(month from order_date)),
    count(distinct(b.book_id)) as cnt_dist
from other.book_orders o 
join other.books b on b.book_id=o.book_id 
where b.publishing_year > 2010
and extract(YEAR FROM o.order_date)>=2019
group by 1
order by 1 desc

`Задание 09-13`

На основе предыдущего запроса создайте новый, чтобы вычислить, сколько раз заказывали каждую книгу в этом месяце.

Столбцы к выводу — `order_month`, `book_name`, `cnt`.

In [ ]:
select 
    extract(month from order_date) as order_month,
    b.book_name as book_name,
    count(b.book_id) as cnt    
from other.book_orders o 
join other.books b on b.book_id=o.book_id 
where b.publishing_year > 2010
and extract(YEAR FROM o.order_date)>=2019
group by 1, 2
order by 1 desc

`Задание 09-14`

Выберите топ-5 книг по заказам в каждом месяце.

Столбцы к выводу — `order_month`, `book_name`, `cnt`, `rnk`.

Отсортируйте вывод по месяцу в обратном порядке и по рангу.

In [ ]:
select 
    extract(month from order_date) as order_month,
    b.book_name as book_name,
    count(b.book_id) as cnt,
    rank() (over partition by b.book_name order by b.book_id desc)    
from other.book_orders o 
join other.books b on b.book_id=o.book_id 
where b.publishing_year > 2010
and extract(YEAR FROM o.order_date)>=2019
and extract(month from order_date) = 1
group by 1, 2
order by 1, 3 desc
limit 5